In [ ]:
# https://leetcode.com/problems/average-time-of-process-per-machine/description/?envType=study-plan-v2&envId=top-sql-50

In [1]:
import pandas as pd


In [3]:
import pandas as pd

# Create a list of data
data = [
    [0, 0, "start", 0.712],
    [0, 0, "end", 1.520],
    [0, 1, "start", 3.140],
    [0, 1, "end", 4.120],
    [1, 0, "start", 0.550],
    [1, 0, "end", 1.550],
    [1, 1, "start", 0.430],
    [1, 1, "end", 1.420],
    [2, 0, "start", 4.100],
    [2, 0, "end", 4.512],
    [2, 1, "start", 2.500],
    [2, 1, "end", 5.000],
]

# Define the column names
col_names = ["machine_id", "process_id", "activity_type", "timestamp"]

# Create the DataFrame
df = pd.DataFrame(data, columns=col_names)

# Print the DataFrame
df.head()


,machine_id,process_id,activity_type,timestamp
0,0,0,start,0.712
1,0,0,end,1.520
2,0,1,start,3.140
3,0,1,end,4.120
4,1,0,start,0.550


In [4]:
df_start = df[df["activity_type"] == "start"]
df_end = df[df["activity_type"] == "end"]

In [6]:
df_merge = pd.merge(df_start, df_end, on=["machine_id", "process_id"], how="inner")

In [7]:
df_merge

,machine_id,process_id,activity_type_x,timestamp_x,activity_type_y,timestamp_y
0,0,0,start,0.712,end,1.520
1,0,1,start,3.140,end,4.120
2,1,0,start,0.550,end,1.550
3,1,1,start,0.430,end,1.420
4,2,0,start,4.100,end,4.512
5,2,1,start,2.500,end,5.000


In [9]:
df_merge = df_merge[["machine_id", "process_id", "timestamp_x", "timestamp_y"]]
df_merge.columns = ["machine_id", "process_id", "timestamp_start", "timestamp_end"]
df_merge


,machine_id,process_id,timestamp_start,timestamp_end
0,0,0,0.712,1.520
1,0,1,3.140,4.120
2,1,0,0.550,1.550
3,1,1,0.430,1.420
4,2,0,4.100,4.512
5,2,1,2.500,5.000


In [10]:
df_merge["duration"] = df_merge["timestamp_end"]  - df_merge["timestamp_start"]
df_merge.head()

,machine_id,process_id,timestamp_start,timestamp_end,duration
0,0,0,0.712,1.520,0.808
1,0,1,3.140,4.120,0.980
2,1,0,0.550,1.550,1.000
3,1,1,0.430,1.420,0.990
4,2,0,4.100,4.512,0.412


In [16]:
df_agg = df_merge.groupby(["machine_id"]).agg({"duration": "mean"}).reset_index()

In [18]:

df_agg.columns=["machine_id", "processing_time"]

In [19]:
df_agg.head()

,machine_id,processing_time
0,0,0.894
1,1,0.995
2,2,1.456


In [20]:
import pandas as pd

def get_average_time(activity: pd.DataFrame) -> pd.DataFrame:
    df = activity
    df_start = df[df["activity_type"] == "start"]
    df_end = df[df["activity_type"] == "end"]
    df_merge = pd.merge(df_start, df_end, on=["machine_id", "process_id"], how="inner")
    df_merge = df_merge[["machine_id", "process_id", "timestamp_x", "timestamp_y"]]
    df_merge.columns = ["machine_id", "process_id", "timestamp_start", "timestamp_end"]
    df_merge["duration"] = df_merge["timestamp_end"]  - df_merge["timestamp_start"]
    df_agg = df_merge.groupby(["machine_id"]).agg({"duration": "mean"}).reset_index()
    df_agg.columns=["machine_id", "processing_time"]
    df_agg = df_agg.round({"processing_time": 3})
    return df_agg

In [ ]:
"""
# Write your MySQL query statement below
SELECT w1.machine_id, ROUND(AVG(w2.timestamp - w1.timestamp), 3) AS processing_time
FROM activity w1
INNER JOIN activity w2 ON w1.machine_id = w2.machine_id AND w1.process_id = w2.process_id
WHERE w1.activity_type = "start" AND w2.activity_type = "end" 
GROUP BY machine_id
"""